In [ ]:
import numpy as np
from unray import *
from ipydatawidgets import *
import pythreejs as three

filename = "data/heart.npz"
#filename = "data/brain.npz"
#filename = "data/aneurysm.npz"

mesh_data = np.load(filename)
cells_array = mesh_data["cells"].astype(np.int32)
points_array = mesh_data["points"].astype(np.float32)

# Coordinates of all vertices in mesh
x = list(points_array.T)  # x[2] = z coordinate array for all vertices

# Model center 3d vector
center = list(map(lambda x: x.mean(), x))

# Model min/max coordinates
bbox = list(map(lambda x: (x.min(), x.max()), x))

# Coordinates with origo shifted to center of model
xm = list(map(lambda x, mp: x - mp, x, center))

# Distance from model center
xd = np.sqrt(sum(map(lambda x: x**2, xm)))
radius = xd.max()

# Distance from center, normalized to max 1.0
func_dist = xd / radius

# A constant for all vertices
func_const = np.ones(x[0].shape)

# x coordinate
func_x = x[0]

# A wave pattern from the center of the model
freq = 4
func_wave = 2.0 + np.sin((freq * 2 * np.pi / radius) * xd)

# Data widgets
cells = NDArrayWidget(cells_array)
points = NDArrayWidget(points_array)
mesh = Mesh(cells=cells, points=points)

field = Field(mesh=mesh, values=func_wave)
lut = ArrayColorLUT(values=[[1,0,0], [0,0,1]])

# pythreejs setup
width = 960
height = 320
camera = three.PerspectiveCamera(position=[10, 10, 10], aspect=width/height)
key_light = three.DirectionalLight(position=[0, 10, 10])
ambient = three.AmbientLight(intensity=0.5)
scene = three.Scene(children=[key_light, ambient, camera], background='gray')
controls = three.OrbitControls(camera)
renderer = three.Renderer(scene, camera, [controls],
                          width=width, height=height)
renderer.background = "#dddddd"
display(renderer)

# Adding some plots
splot = SurfacePlot(mesh=mesh)
scene.add(splot)

In [ ]:
scene.children = tuple(filter(lambda c: c._model_module != "unray", scene.children))

In [ ]:
wp = WireframeParams(color="#000000", opacity=1.0)
splot = SurfacePlot(mesh=mesh, wireframe=wp)
scene.add(splot)

In [ ]:
xplot = XrayPlot(mesh=mesh)
scene.add(xplot)

In [ ]:
plot = SurfacePlot(mesh=mesh)
plot.position = [0, 9, 0]
scene.add(plot)

In [ ]:
splot = SurfacePlot(
    mesh=mesh,
    color=ColorField(field=field, lut=lut),
    wireframe=False,
)
splot.position = [0, 9, 0]
scene.add(splot)

In [ ]:
xplot = XrayPlot(
    density=field,
)
xplot.position = [0, 9, 0]
scene.add(xplot)

In [ ]:
box = three.Mesh(three.BoxGeometry(1, 1, 1), three.MeshLambertMaterial(), position=[5, 0, 0])
scene.add(box)

In [ ]:
plot0 = scene.children[3]
plot1 = scene.children[4]
plot2 = scene.children[6]

In [ ]:
plot0.visible = False
plot1.visible = False

In [ ]:
[(i, type(c)) for i, c in enumerate(scene.children)]